# Flumes: Give your AI a memory 🚀

In just a few cells, you’ll give an agent long-term memory it can **store, search, and summarize** — without duct-taping vector DBs or hacking together RAG code.

This notebook walks you through the basics:
1. **Install** the SDK
2. **Store** a dense, real-world message → watch Flumes extract structured facts & events
3. **Retrieve** those memories to answer a new question

---

## Why Flumes is different

- **One call, many jobs** → extract, store, and assemble context in a single request
- **Smart token budgeting** → predictable caps so prompts don’t blow up costs
- **Entity-first** → memories tied to real users/entities, not just text chunks
- **Human-like retrieval** → balances facts, recency, and graph links out of the box

---

⚡ **Quick start**
You only need your Flumes API key.
The base URL is already built in:  https://api.flumes.ai

In [ ]:
# Paste your API key here or set FLUMES_API_KEY in your environment
import os
API_KEY = os.getenv("FLUMES_API_KEY", "your-api-key")
if API_KEY == "your-api-key":
    print("⚠️ Set FLUMES_API_KEY or edit API_KEY above before running the rest.")


In [ ]:
# Install the SDK (one-time in Colab/local)
%pip install -q openai #Only needed for the advanced agent example
%pip install -q flumes-ai

print("Installed flumes-ai")


In [ ]:
# Initialize the client (entity-first)
from flumes import MemoryClient

client = MemoryClient(api_key=API_KEY, agent_id="onboarding-bot")
entity_id = "user_demo_001"

print("Client ready → entity:", entity_id)


In [ ]:
# Step 1: Add a rich message (Flumes will extract structured facts/events)
# This text is intentionally dense so the graph has plenty to show.

u = client.for_entity(entity_id)

rich_message = (
    "I'm Alice Parker, living in Paris since 2023. I usually travel with my brother Ben. "
    "For work, I'm a product manager at Nova Labs. I prefer museums over nightlife and "
    "love quiet wine bars in the Latin Quarter. My favorite artists are Monet and Van Gogh. "
    "I'm training for a half-marathon; my pace is around 5:30/km. Budget for trips is about €1200. "
    "I have a gluten allergy and avoid crowded places on weekends. The last hotel I loved was Hotel Lumière "
    "near Jardin du Luxembourg; I rated it 5/5 in May 2025. I'm planning a 3-day getaway to Lyon next month, "
    "and I'd like recommendations that include scenic runs by the river, museums, and quiet dinner spots."
)

res = u.add(rich_message, budget="standard")
pack = res.get("pack") or {}
ok_budget = (pack.get("used_tokens", 0) <= pack.get("hard_cap_tokens", 10**9))
print("Stored. request_id=", res.get("request_id"))
print("Pack:", pack)
print("✅ Memories stored and within budget" if ok_budget else "⚠️ Stored but over budget cap")


In [ ]:
# Step 2: Verify memories exist (facts/events)
page = client.get_all(entity_id=entity_id, limit=10)
items = page.get("items", [])
print("Fetched", len(items), "items")
for m in items[:6]:
    print("-", m.get("type"), "::", m.get("text"))
print("✅ Memories listed" if len(items) > 0 else "⚠️ No memories returned")


In [ ]:
# Step 3: Retrieve context for a new message
query = "Looking for a quiet dinner near a scenic running route in Lyon; any museum picks?"
res = client.search(query, entity_id=entity_id, top_k=16, include_scores=True)
matches = res.get("matches", [])
print("Matches:", len(matches))
for m in matches[:6]:
    print(f"- {m.get('text')} (score={round(m.get('score', 0),3)})")
print("✅ Retrieval returned matches" if len(matches) > 0 else "⚠️ No matches yet (indexing may take a few seconds)")


### Next steps
- Try adding another turn (preferences, places, ratings)
- Use the Agent helper for end-to-end chat with citations
- Explore the graph view in the dashboard to see entities and relationships emerge

![Memory Graph from Dashboard](https://raw.githubusercontent.com/alex-n-a/flumes-ai-sdk/main/learning/assets/onboarding-graph.png)

![Filtered Memory Graph from Dashboard](https://raw.githubusercontent.com/alex-n-a/flumes-ai-sdk/main/learning/assets/filtered-onboarding-graph.png)

## Advanced: Retrieval weights, budgeting, and presets

Flumes retrieval combines semantic, keyword (BM25), and graph signals. You control it with a simple `retrieval` object:

- `preset`: one of `balanced | factual | recent | graphy`
- Or `weights`: `{ semantic, bm25, graph_prior, recency_decay, confidence }` (server renormalizes; flags indicate if adjusted)
- `top_k`: number of candidates to return (default 16)

Budgeting keeps cost predictable:
- `budget = "light" | "standard" | "heavy"` → max_context_tokens of ~400 / 1200 / 2400 (hard caps applied)
- Each `add()` returns `pack = { target_tokens, hard_cap_tokens, used_tokens, dropped[] }`

You can pass these knobs per call or set them as defaults on the client.


In [ ]:
# Try custom retrieval weights and a different budget
query = "Quiet dinner near scenic runs and impressionist museums during my trip"
res = client.search(
    query,
    entity_id=entity_id,
    top_k=16,
    weights={"semantic": 0.6, "bm25": 0.3, "graph_prior": 0.1},
    include_scores=True,
)
ms = res.get("matches", [])
print("Top 5 with custom weights:")
for m in ms[:5]:
    print(f"- {m.get('text')} (score={round(m.get('score', 0),3)})")
print("✅ Weighted retrieval returned matches" if len(ms) > 0 else "⚠️ No matches")


## Optional: Agent SDK (end-to-end chat)
If you have an OpenAI key configured, you can run a short chat with grounded context. This uses the same `add()` assemble call under the hood.

Before running the next cells, ensure `OPENAI_API_KEY` is set in your environment.


In [ ]:
# Provide your OpenAI API key explicitly
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-key")
if OPENAI_API_KEY == "your-openai-key":
    raise RuntimeError("Set OPENAI_API_KEY or paste your key in OPENAI_API_KEY before running the Agent example.")

In [ ]:

from flumes import Agent
agent = Agent(
    agent_id="onboarding-bot",
    entity_id=entity_id,
    memory_client=client,
    openai_api_key=OPENAI_API_KEY,
)
answer = agent.chat("Plan my trip.")
print(answer)
print("✅ Agent produced a grounded answer" if answer else "⚠️ Agent returned empty text")


## Wrap‑up and next steps
You're done! You installed the SDK, initialized a client, added a dense message, verified stored memories, and retrieved context.

Where to go next:
- View your memory graph in the dashboard (entities, facts, relationships)
- Read the API reference and concepts (retrieval presets, budgeting, policies)
- Explore SDK examples (Python, Node)

Replace these with your links:
- Dashboard: app.flumes.ai/graph-memory
- Docs: https://docs.flumes.ai/api-reference/system/liveness-&-readiness
- SDK repos: https://github.com/alex-n-a/flumes-ai-sdk
